In [ ]:
import os
import comet_llm

from dotenv import load_dotenv

from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import VectorStoreIndex, PromptTemplate
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.cohere import Cohere

from pinecone import Pinecone

In [ ]:
load_dotenv()

Code below for setting up the index and query engine follows that in the /backend app

In [ ]:
def get_vector_index() -> VectorStoreIndex:
    # Connect to the pinecone index
    pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
    pc_index = pc.Index(os.getenv("PINECONE_INDEX_NAME"))

    # Create vector store from existing index
    vector_store = PineconeVectorStore(pinecone_index=pc_index)
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

    return index

In [ ]:
def configure_llm() -> dict:
    llm_model_name = os.getenv("LLM_MODEL_NAME")
    llm_api_key = os.getenv("COHERE_API_KEY")

    config = {
        "model": llm_model_name,
        "api_key": llm_api_key,
    }

    return config


def configure_embedding_model() -> dict:
    embed_model_name = os.getenv("EMBED_MODEL_NAME")
    embed_api_key = os.getenv("OPENAI_API_KEY")
    embed_dimensions = int(os.getenv("EMBED_DIMENSIONS"))

    config = {
        "model": embed_model_name,
        "api_key": embed_api_key,
        "dimensions": embed_dimensions,
    }

    return config

In [ ]:
def init_settings():
    llm_config = configure_llm()
    embed_config = configure_embedding_model()

    Settings.llm = Cohere(**llm_config)
    Settings.embed_model = OpenAIEmbedding(**embed_config)

In [ ]:
init_settings()

In [ ]:
index = get_vector_index()

In [ ]:
retriever = VectorIndexRetriever(index=index, similarity_top_k=20)


In [ ]:
template = (
    "You are a Protestant theologian, and a careful reader of theology. Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given just this information and no prior knowledge, please answer the question: {query_str}\n"
    "Provide reference citations from the context in your answer. If the provided context uses more than one volume from the Dogmatics, use at least two volumes in your answer."
)

In [ ]:
qa_template = PromptTemplate(template)

In [ ]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [ ]:
query_engine.get_prompts()

In [ ]:
query_engine.update_prompts({
    "response_synthesizer:text_qa_template": qa_template
})

In [ ]:
query_engine.get_prompts()

In [ ]:
query = "How does the threefold office of Jesus relate to his role in election?"

In [ ]:
response = query_engine.query(query)

In [ ]:
response.metadata

In [ ]:
# Given the response.metadata, get the set of volumes in the response
volumes_set = {metadata["volume"] for metadata in response.metadata.values()}
volumes = " ".join(volumes_set)

In [ ]:
response.response

In [ ]:
comet_llm.log_prompt(
    api_key=os.getenv("COMET_API_KEY"),
    project="barth",
    prompt=template,
    prompt_template=template,
    output=response.response,
    prompt_template_variables={"query": query},
    metadata={
        "volumes": volumes
    }
)